# ChurnInsight — Pipeline Principal

Este notebook representa o **pipeline operacional canônico** do projeto **ChurnInsight**.  
Ele consolida, de forma **sequencial, reprodutível e auditável**, todo o fluxo de preparação de dados, definição da variável-alvo, treinamento e geração dos artefatos consumidos pela API de previsão.

Seu papel **não é exploratório**, mas **orquestrador**:  
todas as etapas aqui executadas refletem **decisões técnicas previamente definidas**, documentadas e versionadas nos materiais oficiais do projeto.

📘 **Documentação de referência obrigatória**  
- `pipeline_notebook.md` — descreve a **narrativa operacional** do pipeline, explicando *como* o notebook deve ser lido e interpretado.  
- `pipeline_elements.md` — documenta a **lógica técnica e semântica** de cada elemento exibido no notebook, explicitando contratos, decisões e artefatos produzidos.

Esses documentos complementam este notebook, fornecendo o contexto técnico e semântico das decisões apresentadas ao longo do pipeline.

---

## Estrutura do pipeline

O conteúdo é organizado em **etapas numeradas**, refletindo o fluxo real de transformação dos dados ao longo do projeto:

- Configuração do ambiente e contextualização do projeto  
- Ingestão e caracterização inicial do dataset  
- Diagnóstico de qualidade estrutural e tipagem  
- Conformidade ao contrato e preparação semântica dos dados  
- Tratamento de dados faltantes (com auditoria explícita)  
- Construção e validação da variável-alvo  
- Treinamento e avaliação do modelo  
- Geração dos artefatos finais para integração com a API  

Cada seção segue explicitamente o padrão:

**diagnóstico → decisão explícita → execução → auditoria**

Nenhuma transformação crítica ocorre de forma implícita ou silenciosa.

---

## Autoria e contribuições

Este material foi desenvolvido no contexto das atividades de **Data Science do projeto ChurnInsight**, com as seguintes contribuições:

- **Lilian Moraes** — Cientista de Dados  
- **Lucas Frigato** — Cientista de Dados  
- **Luedji Abayomi** — Cientista de Dados  
- **Fábio Aguiar** — Engenheiro de Dados  


## Contexto de Execução do Notebook
Esta célula estabelece o **contexto de execução** do notebook, tornando explícito o ponto de referência do projeto durante a leitura e execução das etapas seguintes.

Ela atua como um **contrato cognitivo**, ajudando o leitor a compreender de onde os módulos são resolvidos e como o notebook se posiciona dentro da estrutura do projeto, sem impor dependências funcionais ao pipeline.

In [1]:
from pathlib import Path
import sys

ROOT = Path.cwd()
while not (ROOT / "src").exists() and ROOT.parent != ROOT:
    ROOT = ROOT.parent

sys.path.insert(0, str(ROOT))

print("Project ROOT:", ROOT)

Project ROOT: C:\Users\fabio\Projetos DEV\Hackathon ONE\churninsight-hackathon-one


## 1 - Ingestão e diagnóstico inicial
> Esta seção estabelece o estado inicial do dataset após a ingestão, apresentando indicadores puramente diagnósticos de origem, dimensão, tipagem e presença de valores ausentes.
> Nenhuma transformação é aplicada nesta etapa.

In [2]:
from src.data.load_data import list_raw_files
list_raw_files()

,filename,extension,size_bytes,size_human
0,WA_Fn-UseC_-Telco-Customer-Churn.csv,csv,977501,954.59 KB


In [3]:
from src.data.load_data import load_and_report_raw_data
df = load_and_report_raw_data("WA_Fn-UseC_-Telco-Customer-Churn.csv")

## 2 - Qualidade Estrutural & Tipagem
> Esta seção avalia e consolida a qualidade estrutural do dataset, aplicando validações técnicas e conversões de tipo necessárias para garantir consistência operacional.
> As alterações são técnicas e não afetam a semântica de negócio.

In [4]:
from src.data.quality_typing import run_quality_and_typing_report
df = run_quality_and_typing_report(df)

## 3 - Conformidade ao Contrato (API) & Preparação Semântica dos Dados
> Esta seção estabelece o escopo semântico do pipeline, conectando o dataset validado ao contrato formal da API de previsão.
> Aqui são identificadas features, target e candidatos categóricos, sem aplicar transformações.

> Observação:  
> O target (`Churn`) já é fornecido pelo dataset no formato *Yes/No*.  
> Nesta seção, ele é apenas **identificado**, **isolado do escopo de diagnóstico** e **preservado estruturalmente**, sem qualquer transformação ou inferência adicional.

In [5]:
from src.data.contract_loader import load_contract_yaml
from src.features.contract_and_candidates import run_contract_and_candidates
from src.reporting.ui import render_contract_and_candidates_report

CONTRACT_PATH = "contracts/telco_churn.yaml"

cfg = load_contract_yaml(CONTRACT_PATH)
scope = cfg.to_scope()

payload = run_contract_and_candidates(df, scope=scope)
render_contract_and_candidates_report(payload)

Métrica,Antes,Depois,Δ
Linhas,7043,7043,+0.00
Colunas,21,9,-12.00
Memória,6.51 MB,2.77 MB,-3.74 MB


column,dtype,n_unique,pct_unique,sample_values,reasons
OnlineSecurity,object,3,0.0004,"[No, Yes, No internet service]","texto/categoria, frases de serviço"
TechSupport,object,3,0.0004,"[No, Yes, No internet service]","texto/categoria, frases de serviço"
PaperlessBilling,object,2,0.0003,"[Yes, No]","texto/categoria, binário (yes/no)"
Contract,object,3,0.0004,"[Month-to-month, One year, Two year]",texto/categoria
InternetService,object,3,0.0004,"[DSL, Fiber optic, No]",texto/categoria
PaymentMethod,object,4,0.0006,"[Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)]",texto/categoria
column,dtype,n_unique,pct_unique,sample_values,reasons
PaperlessBilling,object,2,0.0003,"[Yes, No]","texto/categoria, binário (yes/no)"
column,dtype,n_unique,pct_unique,sample_values,reasons
OnlineSecurity,object,3,0.0004,"[No, Yes, No internet service]","texto/categoria, frases de serviço"


> Esta próxima etapa executa a padronização categórica declarada explicitamente, derivada do diagnóstico da Seção 3.
> As transformações são irreversíveis no pipeline, restritas às features do contrato e sem aplicação de encoding.

In [6]:
from src.features.categorical_standardization import run_categorical_standardization
from src.reporting.ui import render_categorical_standardization_report

# Nota: a execução aplica normalização textual mínima (strip + lower + colapso de espaços).
S3_CAT_STD_MAP = {"no internet service": "no"}
S3_CAT_STD_COLS = ["OnlineSecurity", "TechSupport"]

payload_std = run_categorical_standardization(
    df=payload["df"],
    scope=payload.get("scope"),
    phrase_map=S3_CAT_STD_MAP,
    cols_scope=S3_CAT_STD_COLS,
)

render_categorical_standardization_report(payload_std)
df = payload_std["df"]


> Esta próxima etapa realiza uma auditoria **somente diagnóstica** da coluna target, verificando
> presença, domínio e consistência básica dos valores, **sem aplicar qualquer transformação**.
> O objetivo é validar se o target está adequado para a etapa de modelagem.

In [7]:
from src.features.target_audit import run_target_audit
from src.reporting.ui import render_target_audit_report

# domínio esperado do Telco churn
EXPECTED_TARGET = ["Yes", "No"]

payload_s33 = run_target_audit(
    df=df,
    scope=scope,
    expected_values=EXPECTED_TARGET
)

render_target_audit_report(payload_s33)

value,count,pct
No,5174,73.46
Yes,1869,26.54


## 4 - Tratamento de Dados Faltantes
> Esta seção executa o tratamento explícito de dados faltantes, por meio de imputação controlada e auditável.
> A execução é irreversível no pipeline, restrita às features do contrato, com preservação explícita do target.

In [8]:
from src.features.missing_imputation import run_missing_imputation
from src.reporting.ui import render_missing_imputation_report

# decision: decisão explícita (SEM defaults escondidos)
decision_s4 = {
    # se None: assume intenção = todas as features (ainda filtradas pelo scope)
    "include_cols": None,
    # opcional: colunas explicitamente excluídas
    "exclude_cols": [],
    # obrigatório
    "numeric_strategy": "median",            # "median" | "mean" | "constant"
    "categorical_strategy": "most_frequent", # "most_frequent" | "constant"
}

payload_s4 = run_missing_imputation(df=df, scope=scope, decision=decision_s4)
df = payload_s4["df"]  # mantém seu padrão de sempre avançar com df
render_missing_imputation_report(payload_s4)

Métrica,Antes,Depois,Δ
Linhas,7043.00,7043.00,0.0
Colunas,9.00,9.00,0.0
Memória (MB),2.72,2.72,0.0
column,kind,strategy,fill_value_used
tenure,numeric,median,29.0
Contract,categorical,most_frequent,Month-to-month
InternetService,categorical,most_frequent,Fiber optic
OnlineSecurity,categorical,most_frequent,no
TechSupport,categorical,most_frequent,no
MonthlyCharges,numeric,median,70.35
